In [2]:
import pandas as pd
from csv import reader 

In [2]:
mapping_df = pd.read_excel('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s4.xls')
mapping_df.head()

,OMIM ID,OMIM disease name,UMLS concept ID,UMLS concept name
0,102100,"Acromegaloid Changes, Cutis Verticis Gyrata, A...",C1868756,Corneal leukoma
1,102100,"Acromegaloid Changes, Cutis Verticis Gyrata, A...",C0263417,Cutis verticis gyrata
2,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome
3,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C1876177,RLS1 (Ekbom Syndrome)
4,102400,Acroosteolysis,C0917990,Acroosteolysis (Acro-Osteolysis)


In [3]:
mapping_df['OMIM disease name'].replace({'Neuropathy, Hereditary Sensory And Autonomic, Type I, With Cough And':
                                         'Neuropathy, Hereditary Sensory And Autonomic, Type I, With Cough And Gastroesophageal Reflux'}, inplace=True)

In [4]:
goldstd_df = pd.read_excel('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s1.xls')
goldstd_df.head()

,Drug name,Disease name
0,Acamprosate,Alcohol Dependence
1,Acarbose,"Diabetes Mellitus, Noninsulin-Dependent; Niddm"
2,Acarbose,"Maturity-Onset Diabetes Of The Young, Type 1; ..."
3,Acarbose,"Maturity-Onset Diabetes Of The Young, Type 2; ..."
4,Acarbose,"Maturity-Onset Diabetes Of The Young, Type 3; ..."


In [5]:
goldstd_df['Drug name'].replace({'Divalproex Sodium':'Valproic Acid',
                                 'Bismuth':'Bismuth subsalicylate',
                                'Clobetasol':'Clobetasol propionate',
                               'Guanadrel Sulfate':'Guanadrel',
                                 'Marinol':'Dronabinol',
                               'Medroxyprogesterone':'Medroxyprogesterone acetate',
                                'Megestrol':'Megestrol acetate',
                                'Propoxyphene':'Dextropropoxyphene',
                                 'Salicyclic Acid':'Salicylic acid',
                                'Ipratropium':'Ipratropiumbromid',
                                'Adenosine Monophosphate':'Adenosine monophosphate',
                                'Arsenic Trioxide':'Arsenic trioxide',
                                'Ethacrynic Acid':'Ethacrynic acid',
                                'Fondaparinux Sodium':'Fondaparinux sodium',
                                 'Meclofenamic Acid':'Meclofenamic acid',
                                'Methyl Aminolevulinate':'Methyl aminolevulinate'},inplace=True)

In [6]:
merged_df = goldstd_df.merge(mapping_df, left_on='Disease name', right_on='OMIM disease name')

In [7]:
merged_df.head()

,Drug name,Disease name,OMIM ID,OMIM disease name,UMLS concept ID,UMLS concept name
0,Acamprosate,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
1,Chlordiazepoxide,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
2,Citalopram,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
3,Disulfiram,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
4,Naltrexone,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."


In [8]:
merged_df.head()

,Drug name,Disease name,OMIM ID,OMIM disease name,UMLS concept ID,UMLS concept name
0,Acamprosate,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
1,Chlordiazepoxide,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
2,Citalopram,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
3,Disulfiram,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."
4,Naltrexone,Alcohol Dependence,103780,Alcohol Dependence,C0001973,"Alcohol Dependence (Alcoholic Intoxication, Ch..."


In [9]:
drug_synonym_df = pd.read_csv('data/input/drugbank-drug-synonym.tab', sep='\t')
drug_synonym_df.head()

,?drugid,?name
0,DB00001,Hirudin variant-1@en
1,DB00002,Anti EGFR@en
2,DB00002,IMC-C225@en
3,DB00003,Deoxyribonuclease I@en
4,DB00003,Deoxyribonuclease-1 precursor@en


In [10]:
drug_synonym_df.rename(columns={'?drugid':'drugid','?name':'name'},inplace=True)

In [11]:
drug_synonym_df.name =drug_synonym_df.name.map(lambda x: x[:-3])

In [12]:
drug_synonym_df.head()

,drugid,name
0,DB00001,Hirudin variant-1
1,DB00002,Anti EGFR
2,DB00002,IMC-C225
3,DB00003,Deoxyribonuclease I
4,DB00003,Deoxyribonuclease-1 precursor


In [13]:
merged_df = merged_df.merge(drug_synonym_df, left_on='Drug name', right_on='name')

In [14]:
len(merged_df[['drugid','OMIM ID']].drop_duplicates())

1933

In [15]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph
def to_rdf(g, df, column_types, row_uri):
    """
    Parameters
    ----------
    g : input rdflib.Graph  
    df: DataFrame to be converted into RDF Graph
    column_types: dictonary of column and its type, type can be URI or Literal
    row_uri: rdf:type value for row index, should be URI
    Returns
    -------
    g: rdflib.Graph generated from DataFrame object
    """
    
    for (index, series) in df.iterrows():
        g.add((URIRef(index), RDF.type, URIRef(row_uri)) )
        for (column, value) in series.iteritems():
            if column_types[column] == 'URI':
                g.add((URIRef(index), URIRef(column), URIRef(value)))
            else:
                g.add((URIRef(index), URIRef(column), Literal(value)))
                
    return g

In [16]:
gold_std_mapped_df = merged_df[['drugid','OMIM ID']].drop_duplicates()

In [17]:
gold_std_mapped_df['drugid'] = gold_std_mapped_df['drugid'].map(lambda x: 'http://bio2rdf.org/drugbank:'+str(x))

In [18]:
gold_std_mapped_df['OMIM ID'] = gold_std_mapped_df['OMIM ID'].map(lambda x: 'http://bio2rdf.org/omim:'+str(x))

In [19]:
gold_std_mapped_df.rename(columns={'OMIM ID':'http://bio2rdf.org/openpredict_vocabulary:indication'},inplace=True)

In [20]:
gold_std_mapped_df= gold_std_mapped_df.set_index('drugid', drop=True)

In [21]:
column_types ={'http://bio2rdf.org/openpredict_vocabulary:indication':'URI'}
g =  ConjunctiveGraph(identifier = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.dataset.openpredict.R1'))     
g=  to_rdf(g, gold_std_mapped_df, column_types, 'http://bio2rdf.org/drugbank:Drug' )

In [22]:
from rdflib import Namespace
import datetime
DC = Namespace("http://purl.org/dc/terms/")
def addProvanace(g):
    now = datetime.datetime.now()
    graphURI = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.dataset.openpredict.R1')
    datasetURI= URIRef('https://github.com/fair-workflows/openpredict/known_associations/predict-gold-standard-omim.nq')
    g.add((graphURI, RDF.type, DC.Dataset))
    g.add((graphURI, URIRef('http://www.w3.org/ns/dcat#distribution'), datasetURI))
   
    
    g.add((datasetURI, DC['title'], Literal('RDF version of PREDICT drug indication gold standard')))
    g.add((datasetURI, DC['format'], Literal('application/n-quads')))
    g.add((datasetURI, DC['created'], Literal(now.strftime("%Y-%m-%d %H:%M:%S"))))
    g.add((datasetURI, DC['creator'], Literal('https://github.com/fair-workflows/openpredict/MappingPREDICTGoldstandard.ipynb')))
    g.add((datasetURI, DC['source'], URIRef('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s1.xls')))
    g.add((datasetURI, DC['source'], URIRef('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s4.xls')))

    g.add((datasetURI, DC['homepage'], URIRef('https://github.com/fair-workflows/openpredict/')))
    g.add((datasetURI, DC['license'], URIRef('http://creativecommons.org/licenses/by/3.0/')))
    g.add((datasetURI, DC['rights'], Literal('use-share-modify')))
    g.add((datasetURI, DC['rights'], Literal('by-attribution')))
    g.add((datasetURI, DC['rights'], Literal('restricted-by-source-license')))
        
    sourcedatasetURI = URIRef('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s1.xls')
    g.add((sourcedatasetURI, DC['title'], Literal('Drug indications gold standard used in the PREDICT study (msb201126-s1.xls)')))
    g.add((sourcedatasetURI, RDF['type'], URIRef('http://www.w3.org/ns/dcat#Distribution')))
    g.add((sourcedatasetURI, DC['homepage'], URIRef('https://dx.doi.org/10.1038%2Fmsb.2011.26')))
    g.add((sourcedatasetURI, URIRef('http://purl.org/pav/retrievedOn'), Literal(now.strftime("%Y-%m-%d %H:%M:%S"))))
    g.add((sourcedatasetURI, DC['format'], Literal('application/xls')))
    g.add((sourcedatasetURI, DC['rights'], URIRef('https://creativecommons.org/licenses/by-nc-sa/3.0/us/')))
    g.add((sourcedatasetURI, DC['publisher'], Literal('Molecular Systems Biology')))
    g.add((sourcedatasetURI, DC['rights'], Literal('use')))
    g.add((sourcedatasetURI, DC['rights'], Literal('no-commercial')))
          

    sourcedatasetURI = URIRef('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3159979/bin/msb201126-s4.xls')
    g.add((sourcedatasetURI, DC['title'], Literal('Mapping between OMIM diseases and UMLS concepts used in the PREDICT study (msb201126-s4.xls)')))
    g.add((sourcedatasetURI, RDF['type'], URIRef('http://www.w3.org/ns/dcat#Distribution')))
    g.add((sourcedatasetURI, DC['homepage'], URIRef('https://dx.doi.org/10.1038%2Fmsb.2011.26')))
    g.add((sourcedatasetURI, URIRef('http://purl.org/pav/retrievedOn'), Literal(now.strftime("%Y-%m-%d %H:%M:%S"))))
    g.add((sourcedatasetURI, DC['format'], Literal('application/xls')))
    g.add((sourcedatasetURI, DC['rights'], URIRef('https://creativecommons.org/licenses/by-nc-sa/3.0/us/')))
    g.add((sourcedatasetURI, DC['publisher'], Literal('Molecular Systems Biology')))
    g.add((sourcedatasetURI, DC['rights'], Literal('use')))
    g.add((sourcedatasetURI, DC['rights'], Literal('no-commercial')))
    
    return g

In [23]:
g= addProvanace(g)

In [24]:
g.serialize('data/rdf/predict-gold-standard-omim.nq', format='nquads')